In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    # df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)

    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
        )])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-31 01:55:06.774409: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 01:55:06.806441: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 01:55:06.806478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 01:55:06.807566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 01:55:06.813235: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 01:55:06.813763: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
from keras import saving
from keras import models
import pickle

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, model_boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ Dataset [{idx + 1}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= model_boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  # brief: for each dataset
  def LoopDataset(self, sort_by, model_boundarie):
    for j in range(self.dataset_run_times):
      trainer = self.PrepareTraining() # split dataset
      if (self.LoopWeights(sort_by, model_boundarie, trainer, j) == True):
        self.SaveDataset(trainer, f"dataset_{j}")
        self.DisplayBetterResults('mse_sup', j)
      self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    excel_path = f'../results/metrics_{dataset}.xlsx'
    df.to_excel(excel_path, index=True)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [8]:
tester = Tester(run_times=100, dataset_run_times=50)

In [9]:
tester.LoopDataset(sort_by='mse_sup', model_boundarie=2.5)

+++++++++++ Dataset [1] | 1 ++++++++++++++++++
Elapsed time:  0.8251081240000531
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ Dataset [1] | 2 ++++++++++++++++++
Elapsed time:  0.8447732839999844
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ Dataset [1] | 3 ++++++++++++++++++
Elapsed time:  0.5228204530003495
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 4 ++++++++++++++++++
Elapsed time:  0.4526237969998874
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 5 ++++++++++++++++++
Elapsed time:  0.5038309990000016
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 6 ++++++++++++++++++
Elapsed time:  0.4488009599999714
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 7 ++++++++++++++++++
Elapsed time:  0.440397763999

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_6_1,0.877306,0.659463,0.611560,0.574052,0.709155,0.514920,2.277170,0.684458,2.377856,1.531157
model_6_0,0.860573,0.646993,0.968428,0.499253,0.729220,0.585145,2.360561,0.055631,2.795423,1.425527
model_6_2,0.876171,0.636792,0.837052,0.502170,0.708780,0.519684,2.428775,0.287125,2.779139,1.533132
model_6_5,0.821511,0.631068,0.258345,0.427733,0.572465,0.749078,2.467051,1.306848,3.194684,2.250765
model_6_3,0.867129,0.630885,0.785386,0.511559,0.705111,0.557633,2.468277,0.378165,2.726727,1.552447
model_6_4,0.857221,0.629663,0.832038,0.545811,0.731079,0.599213,2.476442,0.295960,2.535512,1.415736


DataFrame salvo em ../results/metrics_6.xlsx
+++++++++++ Dataset [8] | 1 ++++++++++++++++++
Elapsed time:  3.3032061769999927
Stopped at epoch:  86
1/1 [==============================] - 0s 20ms/step
+++++++++++ Dataset [8] | 2 ++++++++++++++++++
Elapsed time:  1.9559924659997705
Stopped at epoch:  47
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [8] | 3 ++++++++++++++++++
Elapsed time:  2.715361285000199
Stopped at epoch:  67
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [8] | 4 ++++++++++++++++++
Elapsed time:  4.107061754999904
Stopped at epoch:  114
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [8] | 5 ++++++++++++++++++
Elapsed time:  3.094566354000108
Stopped at epoch:  79
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [8] | 6 ++++++++++++++++++
Elapsed time:  2.1680504009996184
Stopped at epoch:  57
1/1 [==============================] - 0s 20ms/step
+++++++++++ Dataset [8] | 7

2024-01-31 02:25:05.862911: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  7.616109391000464
Stopped at epoch:  207
1/1 [==============================] - 0s 26ms/step
+++++++++++ Dataset [9] | 47 ++++++++++++++++++
Elapsed time:  2.848267321000094
Stopped at epoch:  75
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [9] | 48 ++++++++++++++++++
Elapsed time:  2.7389492410002276
Stopped at epoch:  75
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [9] | 49 ++++++++++++++++++
Elapsed time:  3.779887527000028
Stopped at epoch:  111
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [9] | 50 ++++++++++++++++++
Elapsed time:  4.6357520450001175
Stopped at epoch:  131
1/1 [==============================] - 0s 18ms/step
+++++++++++ Dataset [9] | 51 ++++++++++++++++++
Elapsed time:  2.950166230000832
Stopped at epoch:  77
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [9] | 52 ++++++++++++++++++
Elapsed time:  3.818850972999826
Stopped at epoch:  95
1/1 [=====

...

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_46_7,0.870310,0.732543,0.121699,0.688159,0.648885,0.544281,1.788485,0.985240,2.204409,1.594825
model_46_10,0.812652,0.691462,-0.342129,0.555141,0.488102,0.786260,2.063196,1.505542,3.144715,2.325129
model_46_4,0.867561,0.684766,0.798196,0.684461,0.729543,0.555820,2.107974,0.226375,2.230551,1.228463
model_46_11,0.823867,0.652410,0.727857,0.655439,0.698274,0.739194,2.324339,0.305278,2.435707,1.370493
model_46_2,0.823143,0.650214,0.653123,0.586129,0.635112,0.742229,2.339024,0.389112,2.925660,1.657386
model_46_3,0.858171,0.649147,0.818740,0.652940,0.707551,0.595224,2.346159,0.203329,2.453374,1.328352
model_46_6,0.829495,0.648624,0.860430,0.602146,0.673174,0.715573,2.349653,0.156563,2.812441,1.484502
model_46_5,0.852363,0.646400,0.853554,0.648705,0.708555,0.619602,2.364522,0.164277,2.483314,1.323796
model_46_1,0.857654,0.645783,0.811739,0.717503,0.756927,0.597394,2.368653,0.211183,1.996974,1.104079
model_46_8,0.832634,0.639735,0.850491,0.655160,0.713199,0.702398,2.409096,0.167712,2.437684,1.302698


DataFrame salvo em ../results/metrics_46.xlsx
+++++++++++ Dataset [48] | 1 ++++++++++++++++++
Elapsed time:  1.491415047999908
Stopped at epoch:  34
1/1 [==============================] - 0s 21ms/step
+++++++++++ Dataset [48] | 2 ++++++++++++++++++
Elapsed time:  2.2903752610000083
Stopped at epoch:  58
1/1 [==============================] - 0s 23ms/step
+++++++++++ Dataset [48] | 3 ++++++++++++++++++
Elapsed time:  0.7171589890003816
Stopped at epoch:  7
1/1 [==============================] - 0s 19ms/step
+++++++++++ Dataset [48] | 4 ++++++++++++++++++
Elapsed time:  3.8180206140004884
Stopped at epoch:  97
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [48] | 5 ++++++++++++++++++
Elapsed time:  2.876651820999541
Stopped at epoch:  80
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [48] | 6 ++++++++++++++++++
Elapsed time:  0.842868459998499
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [48

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_47_0,0.834601,0.634789,-0.14376,0.71267,0.573752,0.694143,2.44217,1.088348,1.411882,1.250115


DataFrame salvo em ../results/metrics_47.xlsx
+++++++++++ Dataset [49] | 1 ++++++++++++++++++
Elapsed time:  0.6596330609991128
Stopped at epoch:  7
1/1 [==============================] - 0s 23ms/step
+++++++++++ Dataset [49] | 2 ++++++++++++++++++
Elapsed time:  0.6365548030007631
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [49] | 3 ++++++++++++++++++
Elapsed time:  1.1712877600002685
Stopped at epoch:  19
1/1 [==============================] - 0s 19ms/step
+++++++++++ Dataset [49] | 4 ++++++++++++++++++
Elapsed time:  1.1290609660009068
Stopped at epoch:  21
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [49] | 5 ++++++++++++++++++
Elapsed time:  0.809482325999852
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [49] | 6 ++++++++++++++++++
Elapsed time:  1.1187725929994485
Stopped at epoch:  22
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [4

: 